In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import pandas as pd

import kraft

In [ ]:
function = kraft.compute_ic

In [ ]:
def simulate_vector_and_matrix(shape, matrix_n_category=None):

    if matrix_n_category is None:

        vector = np.random.normal(size=shape[1])

        matrix = np.random.normal(size=shape)

    else:

        vector = np.random.randint(0, high=matrix_n_category, size=shape[1])

        matrix = np.random.randint(0, high=matrix_n_category, size=shape)

    columns = pd.Index(("Column{}".format(i) for i in range(shape[1])), name="Column")

    return (
        pd.Series(vector, index=columns, name="Vector"),
        pd.DataFrame(
            matrix,
            index=pd.Index(("Row{}".format(i) for i in range(shape[0])), name="Row"),
            columns=columns,
        ),
    )

In [ ]:
for shape in (
    (1, 8),
    (2, 8),
    (4, 8),
    (8, 8),
    (16, 8),
    (32, 8),
    (64, 8),
    (128, 8),
    (256, 8),
    (8, 2),
    (8, 4),
    (8, 16),
    (8, 32),
    (8, 64),
    (8, 128),
    (8, 256),
):

    vector, matrix = simulate_vector_and_matrix(shape)

    kraft.function_heat_map(vector, matrix, function, n_extreme=None)

In [ ]:
shape = (10, 20)

vector, matrix = simulate_vector_and_matrix(shape)

# matrix.iloc[0] = np.nan

matrix.iloc[:, 0] = np.nan

matrix.iloc[-2:, -2:] = np.nan

kraft.function_heat_map(vector, matrix, function)

In [ ]:
shape = (10, 8)

matrix = simulate_vector_and_matrix(shape)[1]

# matrix.iloc[0] = np.nan

matrix.iloc[:, 0] = np.nan

matrix.iloc[-2:, -2:] = np.nan

for vector in (
    ((1, 2, 2, 2, 2, 0, 0, 0)),
    ((1, 1, 2, 2, 2, 0, 0, 0)),
):

    vector = pd.Series(vector, index=matrix.columns)

    for vector_ascending in (None, True, False):

        kraft.function_heat_map(
            vector,
            matrix,
            function,
            vector_ascending=vector_ascending,
            vector_data_type="categorical",
        )

In [ ]:
vector = pd.read_csv(
    "function_heat_map_vector.tsv", sep="\t", index_col=0, squeeze=True
)

matrix = pd.read_csv("function_heat_map_matrix.tsv", sep="\t", index_col=0)

kraft.function_heat_map(
    vector, matrix, function, n_sampling=0, n_permutation=0, vector_data_type="binary"
)

kraft.function_heat_map(
    matrix.iloc[0], matrix.iloc[:1], function, n_sampling=0, n_permutation=0, plot_std=3
)

kraft.function_heat_map(
    matrix.iloc[0], matrix.iloc[:2], function, n_sampling=0, n_permutation=0, plot_std=3
)

In [ ]:
vector, matrix = simulate_vector_and_matrix((4, 10))

matrix_3 = simulate_vector_and_matrix((3, 10), 3)[1]

matrix_2 = simulate_vector_and_matrix((2, 10), 2)[1]

matrix.drop(matrix.columns[-3], axis=1, inplace=True)

matrix_3.drop(matrix_3.columns[-2], axis=1, inplace=True)

matrix_2.drop(matrix_2.columns[-1], axis=1, inplace=True)

matrix_dicts = {
    "Matrix A": {"matrix": matrix, "data_type": "continuous"},
    "Matrix B": {"matrix": matrix_3, "data_type": "categorical"},
    "Matrix C": {"matrix": matrix_2, "data_type": "binary"},
}

statistics_dicts = {
    name: kraft.function_heat_map(vector, matrix_dict["matrix"], function, plot=False)
    for name, matrix_dict in matrix_dicts.items()
}

for plot_only_shared in (False, True):

    kraft.function_heat_map_summary(
        vector, matrix_dicts, statistics_dicts, plot_only_shared=plot_only_shared,
    )